<a href="https://colab.research.google.com/github/ICam99/roboticaMovil/blob/main/Tarea3_IgnacioC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from IPython.display import display, HTML
import numpy as np
import matplotlib.pyplot as plt
import random
import base64
import imageio
import os

# Tamaño del espacio
width, height = 6.3, 6.3

# Posición inicial y meta
start = (6, 6)  # Cruz roja
goal = (0, 0)   # Cruz verde

# Parámetros del RRT
step_size = 0.5  # Tamaño de cada paso
max_nodes = 1000  # Número máximo de nodos
goal_threshold = 1  # Margen de tolerancia para considerar que se llegó a la meta

# Definir obstáculos
grid_x = np.linspace(1.5, 5.5, 5)
grid_y = np.linspace(1.5, 5.5, 5)
obstacles = {(round(x, 1), round(y, 1)) for x in grid_x for y in grid_y}

# Clase para representar un nodo del árbol RRT
class Node:
    def __init__(self, pos, parent=None, angle=0):
        self.pos = pos
        self.parent = parent
        self.angle = angle

# Función para obtener un punto aleatorio en el espacio
def random_point():
    return (random.uniform(0, width), random.uniform(0, height))

# Función para encontrar el nodo más cercano en el árbol
def nearest_node(tree, point):
    return min(tree, key=lambda node: np.linalg.norm(np.array(node.pos) - np.array(point)))

# Función para dibujar la cruz
def draw_rotated_cross(ax, pos, angle, size=0.5, color='blue'):
    x, y = pos
    dx = size / 2 * np.cos(angle)
    dy = size / 2 * np.sin(angle)

    ax.plot([x - dx, x + dx], [y - dy, y + dy], color=color, linewidth=3)
    ax.plot([x - dy, x + dy], [y + dx, y - dx], color=color, linewidth=3)

# Función para mover un nodo hacia un nuevo punto
def steer(from_node, to_point, step_size):
    direction = np.array(to_point) - np.array(from_node.pos)
    distance = np.linalg.norm(direction)
    if distance < step_size:
        new_pos = to_point
    else:
        new_pos = np.array(from_node.pos) + (direction / distance) * step_size
        new_pos = tuple(new_pos)

    # Calcula el ángulo de rotación
    angle = np.arctan2(direction[1], direction[0])

    # Evitar obstáculos
    for obstacle in obstacles:
        if np.linalg.norm(np.array(new_pos) - np.array(obstacle)) < 0.4:
            return None

    return Node(new_pos, from_node, angle)

# Función para verificar si se ha alcanzado la meta
def reached_goal(node, goal, threshold=goal_threshold):
    return np.linalg.norm(np.array(node.pos) - np.array(goal)) < threshold

# Implementación de RRT evitando obstáculos
def rrt(start, goal, step_size, max_nodes):
    tree = [Node(start)]
    for _ in range(max_nodes):
        rand_point = random_point()
        nearest = nearest_node(tree, rand_point)
        new_node = steer(nearest, rand_point, step_size)
        if new_node:
            tree.append(new_node)
            if reached_goal(new_node, goal):
                return tree, new_node  # Ruta encontrada
    return tree, None  # No encontró ruta

# Ejecutar RRT
tree, final_node = rrt(start, goal, step_size, max_nodes)
print(f"Tree length: {len(tree)}")

# Reconstruir la trayectoria desde la meta hasta el inicio
path = []
if final_node:
    node = final_node
    while node:
        path.append((node.pos, node.angle))
        node = node.parent
    path.reverse()

if not path:
    print("⚠️ No se encontró un camino hacia la meta.")

print(f"Path length: {len(path)}")

frames_dir = "frames"
if not os.path.exists(frames_dir):
    os.makedirs(frames_dir)

# Función para dibujar cada frame
def draw_frame(frame_num, current_pos, current_angle):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)

    # Dibujar el fondo
    ax.plot(start[0], start[1], 'r+', markersize=30, label="Inicio (Cruz Roja)")
    ax.plot(goal[0], goal[1], 'g+', markersize=30, label="Meta (Cruz Verde)")

    # Dibujar obstáculos (puntos negros)
    obstacle_x, obstacle_y = zip(*obstacles)
    ax.scatter(obstacle_x, obstacle_y, s=100, color='black', label="Obstáculos")

    # Dinujar la cruz
    draw_rotated_cross(ax, current_pos, current_angle)

    ax.legend()
    plt.savefig(f"{frames_dir}/frame_{frame_num:04d}.png")
    plt.close(fig)

# Generar frames
if path:
    print("Si se encontró un camino hacia la meta.")
    for i, (pos, angle) in enumerate(path):
        draw_frame(i, pos, angle)

    # Crea el video a partir de los frames
    images = []
    for i in range(len(path)):
        filename = f"{frames_dir}/frame_{i:04d}.png"
        images.append(imageio.imread(filename))
    imageio.mimsave('rrt_animation.mp4', images, fps=10)

    for filename in os.listdir(frames_dir):
        file_path = os.path.join(frames_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    os.rmdir(frames_dir)

    # Muestra el video creado
    video = open('rrt_animation.mp4', 'rb').read()
    data_url = "data:video/mp4;base64," + base64.b64encode(video).decode()
    display(HTML(f'<video width=400 controls><source src="{data_url}" type="video/mp4"></video>'))
else:
    print("⚠️ No se encontró un camino hacia la meta.")

Tree length: 120
Path length: 23
Si se encontró un camino hacia la meta.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-24-b667c8c8b2bd>:144: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
